<a href="https://colab.research.google.com/github/kviercz/AAI_500_Group_Project/blob/main/kim_assignment2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


In [18]:
# 1. Load Dataset
df = pd.read_csv('M2-Advertising Dataset.csv')

In [7]:
# prompt: print out the columns

print(df.columns.to_list())


['Daily Time Spent on Site', 'Age', 'Area Income', 'Daily Internet Usage', 'Ad Topic Line', 'City', 'Male', 'Country', 'Clicked on Ad']


In [19]:
# 2. Data preprocessing

# Get info about the dataset and check for null values
print(df.describe())
df.info()
df.isnull().sum()

# Convert the countries column into categorical values to improve data training
df = pd.get_dummies(df, columns=['Country'], drop_first=True)

# Try to use data processing to handle the ad topic
tfidf = TfidfVectorizer(stop_words='english', max_features=50)  # Limit to top 50 features
ad_topic_features = tfidf.fit_transform(df['Ad Topic Line']).toarray()
ad_topic_df = pd.DataFrame(ad_topic_features, columns=tfidf.get_feature_names_out())

# Concatenate TF-IDF features with the original dataframe
df = pd.concat([df, ad_topic_df], axis=1)

# Drop the original 'Ad Topic Line' column
df = df.drop(columns=['Ad Topic Line'])

# Handle non-numeric columns
df.drop(columns=['Timestamp'], inplace=True)
df = pd.get_dummies(df, columns=['City'])


       Daily Time Spent on Site          Age   Area Income  \
count               1000.000000  1000.000000   1000.000000   
mean                  65.000200    36.009000  55000.000080   
std                   15.853615     8.785562  13414.634022   
min                   32.600000    19.000000  13996.500000   
25%                   51.360000    29.000000  47031.802500   
50%                   68.215000    35.000000  57012.300000   
75%                   78.547500    42.000000  65470.635000   
max                   91.430000    61.000000  79484.800000   

       Daily Internet Usage         Male  Clicked on Ad  
count           1000.000000  1000.000000     1000.00000  
mean             180.000100     0.481000        0.50000  
std               43.902339     0.499889        0.50025  
min              104.780000     0.000000        0.00000  
25%              138.830000     0.000000        0.00000  
50%              183.130000     0.000000        0.50000  
75%              218.792500     1.0

In [20]:
# 3. Split data into train and test sets
X = df.drop('Clicked on Ad', axis=1)
y = df['Clicked on Ad']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [21]:
# 4. Scale the data using Standard Scaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [25]:
# 5. Build an ANN Classification Model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=[X_train.shape[1]]),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(1)
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=16)
test_loss, test_accuracy = model.evaluate(X_test, y_test)
# print('Test Loss: {:.4f}'.format(test_loss))

Epoch 1/100
50/50 [==============================] - 2s 4ms/step - loss: 4.6554 - accuracy: 0.5238
Epoch 2/100
50/50 [==============================] - 0s 4ms/step - loss: 2.7449 - accuracy: 0.6650
Epoch 3/100
50/50 [==============================] - 0s 3ms/step - loss: 1.5122 - accuracy: 0.7600
Epoch 4/100
50/50 [==============================] - 0s 4ms/step - loss: 0.8608 - accuracy: 0.8537
Epoch 5/100
50/50 [==============================] - 0s 4ms/step - loss: 0.7102 - accuracy: 0.8900
Epoch 6/100
50/50 [==============================] - 0s 3ms/step - loss: 0.4735 - accuracy: 0.9287
Epoch 7/100
50/50 [==============================] - 0s 3ms/step - loss: 0.4150 - accuracy: 0.9400
Epoch 8/100
50/50 [==============================] - 0s 2ms/step - loss: 0.2461 - accuracy: 0.9575
Epoch 9/100
50/50 [==============================] - 0s 2ms/step - loss: 0.2344 - accuracy: 0.9675
Epoch 10/100
50/50 [==============================] - 0s 2ms/step - loss: 0.2241 - accuracy: 0.9663
Epoch 11/

In [30]:
# 7 Model's performance
print('Test Loss: {:.4f}'.format(test_loss))
print('Test Accuracy: {:.4f}'.format(test_accuracy))

# ROC AUC
y_pred_prob = model.predict(X_test)
roc_auc = roc_auc_score(y_test, y_pred_prob)
print('ROC AUC: {:.4f}'.format(roc_auc))

# Predictive classes
y_pred = (y_pred_prob > 0.5).astype(int)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: {:.4f}'.format(accuracy))

# Precision
precision = precision_score(y_test, y_pred)
print('Precision: {:.4f}'.format(precision))

# Recall
recall = recall_score(y_test, y_pred)
print('Recall: {:.4f}'.format(recall))

# F1
f1 = f1_score(y_test, y_pred)
print('F1 Score: {:.4f}'.format(f1))

Test Loss: 2.0829
Test Accuracy: 0.7900
7/7 [==============================] - 0s 3ms/step
ROC AUC: 0.8684
Accuracy: 0.7900
Precision: 0.7273
Recall: 0.8696
F1 Score: 0.7921
